<a href="https://colab.research.google.com/github/rahulsm27/ML/blob/main/Transformer_Practicse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Chapter 2 Text Classification

In [ ]:
# Transformer practicse